# Optional: Difference of Squares Implementation

For Math 173A, you're expected to know the Difference of Squares factorization algorithm, but you're not expected to know how to implement it in Python, so only look at this if you're interested or if you think it will help you understand the algorithm better.

The galois package is used for working over finite fields.  We want to work over $\mathbb{Z}/2\mathbb{Z}$.

In [ ]:
!pip install galois pycryptodome

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 78.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 101.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 85.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 51.7 MB/s eta 0:00:00
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [ ]:
from math import gcd, sqrt, floor
from sympy import factorint, isprime
from Crypto.Util.number import getPrime
import galois
import pandas as pd
import numpy as np

Here is a sample RSA modulus `N` we will try to factor.  The same one from lecture on Week 5 Friday.  The point is to demonstrate the algorithm from Hoffstein, Pipher, Silverman, Section 3.6.  This `N` is small enough we could easily factor it using brute force.

In [ ]:
N = 914387

A number is $L$-smooth if its biggest prime divisor $p$ satisfies $p \leq L$.  For example, $4400 = 2^6 \cdot 3 \cdot 5^2$, so this is $5$-smooth but not $4$-smooth.

In [ ]:
factorint(4800)

{2: 6, 3: 1, 5: 2}

In [ ]:
# Biggest prime divisor
max(factorint(4800))

5

In [ ]:
L = 12
primes = [x for x in range(L+1) if isprime(x)]
primes

[2, 3, 5, 7, 11]

We'll store the results in a matrix, like in the Week 5 Friday lecture.

In [ ]:
df = pd.DataFrame(index = primes)

We look for smooth numbers among the reductions $x^2 \bmod N$.  We'll never get anything interesting unless $x^2 > N$, so that is why we start at $x = \lfloor \sqrt{N} \rfloor + 1$.

In [ ]:
start = floor(sqrt(N)) + 1
for x in range(start, 5000):
    factor_dct = factorint(pow(x,2,N))
    # go to next x if x^2 mod N is not L-smooth
    if max(factor_dct) > L:
        continue
    # Default exponent of 0
    df[x] = 0
    df[x].update(factor_dct)

In [ ]:
df

,1869,1909,3387,4586
2,4,14,0,0
3,1,0,1,2
5,6,1,3,1
7,0,0,0,0
11,0,1,3,1


We need to make two changes before we can use the "linear algebra over finite fields" capabilities of the `galois` library.

1.  We need to switch from a pandas DataFrame to a NumPy array.
2.  We need to reduce the entries.

In [ ]:
# This is how we indicate we want to work mod 2.
GF = galois.GF(2)

In [ ]:
# Doesn't accept a pandas DataFrame
GF(df)

TypeError: GF(2) arrays can be created with scalars of type int/str, lists/tuples, or ndarrays, not <class 'pandas.core.frame.DataFrame'>.

In [ ]:
# Needs entries 0 or 1
GF(df.values)

ValueError: GF(2) arrays must have elements in `0 <= x < 2`, not [ 4 14  2  6  3  3].

In [ ]:
mat = GF((df.values)%2)

In [ ]:
mat

GF([[0, 0, 0, 0],
    [1, 0, 1, 0],
    [0, 1, 1, 1],
    [0, 0, 0, 0],
    [0, 1, 1, 1]], order=2)

In [ ]:
ns = mat.null_space()

In [ ]:
# Basis for the null-space.
ns

GF([[1, 0, 1, 1],
    [0, 1, 0, 1]], order=2)

In [ ]:
# Focus on the second basis vector
vec = ns[1]
vec

GF([0, 1, 0, 1], order=2)

In [ ]:
# x values corresponding to vec
cols = df.columns[vec == 1]
cols

Int64Index([1909, 4586], dtype='int64')

In [ ]:
# prime exponents corresponding to vec
prime_series = df[cols].sum(axis=1)
prime_series

2     14
3      2
5      2
7      0
11     2
dtype: int64

In [ ]:
a = np.prod(cols)
b = np.prod(prime_series.index ** (prime_series.values//2))

In [ ]:
# Success
gcd(a-b, N)

1103

Let's try with a bigger value of `N`, but we're still in the range where the values can be factored naively.

In [ ]:
p = getPrime(13)
q = getPrime(13)
N = p*q
print(N)
print(p)
print(q)

44762551
6343
7057


Here are the values we used:
`N = 44762551`
`p = 6343`
`q = 7057`

In [ ]:
L = 80
primes = [x for x in range(L+1) if isprime(x)]

In [ ]:
df = pd.DataFrame(index = primes)

In [ ]:
start = floor(sqrt(N)) + 1
for x in range(start, 10000):
    factor_dct = factorint(pow(x,2,N))
    # go to next x if x^2 mod N is not L-smooth
    if max(factor_dct) > L:
        continue
    # Default exponent of 0
    df[x] = 0
    df[x].update(factor_dct)

In [ ]:
df.shape

(22, 50)

In [ ]:
mat = GF((df.values)%2)

In [ ]:
ns = mat.null_space()

In [ ]:
for vec in ns:
    cols = df.columns[vec == 1]
    prime_series = df[cols].sum(axis=1)
    a = np.prod(cols)
    b = np.prod(prime_series.index ** (prime_series.values//2))
    g = gcd(a-b, N)
    print(g)

1
1
1
1
6343
44762551
6343
1
1
1
1
44762551
1
6343
1
6343
1
1
6343
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
44762551
/tmp/ipykernel_35/329814783.py:6: RuntimeWarning: overflow encountered in long_scalars
  g = gcd(a-b, N)


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=8699e658-c5b9-40c8-917b-2c7e6a1d0db8' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>